In [20]:
import json

In [2]:
ls *.json

 O volume na unidade C é OS
 O Número de Série do Volume é 1299-FF61

 Pasta de C:\Users\danie\Documents\OncoData_roraima\dsmil_stomach

19/02/2024  20:57           178.769 stomach_json.json
               1 arquivo(s)        178.769 bytes
               0 pasta(s)   152.917.454.848 bytes disponíveis


In [57]:
INPUT_FILENAME = 'stomach_json.json'
INPUT_JSON = json.loads(open(INPUT_FILENAME, 'r').read())
DIVISION_TRAIN = 0.7
DESIRED_TARGETS = ['Primary Tumor', 'Solid Tissue Normal']

In [58]:
for input in INPUT_JSON:
    input['image_path'] = input['image_path'].replace('D:\\Stomach_tissue\\', '/tmp/train-data/').replace('\\', '/')

In [59]:
INPUT_JSON[0]

{'image_tag': 'Primary Tumor',
 'image_path': '/tmp/train-data/f13da9d5-8de5-477d-b160-b1f8ae8e4adf/TCGA-CG-4476-01A-01-TS1.445551b3-54ac-4d7b-80b9-23bc0ecb3812.svs',
 'dataset': 'Primary Tumor',
 'participant': '4476',
 'size_gb': 0.060468733,
 'size_img': [16000, 12776]}

In [60]:
TARGET_JSON = [i for i in INPUT_JSON if i['image_tag'] in DESIRED_TARGETS]

In [61]:
len(TARGET_JSON), len(INPUT_JSON)

(532, 532)

In [62]:
def divide_by_participant(input_json):
    grouped = {}
    for i in input_json:
        p = i['participant']
        if p in grouped:
            grouped[p].append(i)
        else:
            grouped[p] = [i]
    return grouped

In [63]:
grouped = divide_by_participant(TARGET_JSON)

In [64]:
def make_division(grouping, train_perc):
    train = []
    test = []
    divide_by_size = {}
    for key in grouping.keys():
        num_imgs = len(grouping[key])
        if num_imgs in divide_by_size:
            divide_by_size[num_imgs].append(key)
        else:
            divide_by_size[num_imgs] = [key]
    
    for key in divide_by_size.keys():
        num_keys = len(divide_by_size[key])
        train_keys = divide_by_size[key][:int(train_perc*num_keys)]
        test_keys = divide_by_size[key][int(train_perc*num_keys):]
        
        for sub_key in train_keys:
            train += grouping[sub_key]
        for sub_key in test_keys:
            test += grouping[sub_key]
    return train, test

In [65]:
train_data, test_data = make_division(grouped, DIVISION_TRAIN)

In [66]:
len(train_data)/(len(train_data) + len(test_data))

0.6898496240601504

In [67]:
len([t for t in train_data if t['image_tag'] == 'Solid Tissue Normal'])

60

In [68]:
len([t for t in train_data if t['image_tag'] == 'Primary Tumor'])

307

In [69]:
out = open(INPUT_FILENAME.replace('.json', '') + '_train.json', 'w')
out.write(json.dumps(train_data, indent=2))
out.close()

In [70]:
out = open(INPUT_FILENAME.replace('.json', '') + '_test.json', 'w')
out.write(json.dumps(test_data, indent=2))
out.close()